<a href="https://colab.research.google.com/github/bhowad-akash/Data-Analysis-with-Pandas/blob/main/Driver_Inattention_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Setting paths
base_dir_path = '/content/drive/MyDrive/Machine Learning Project/Driver Inattention Detection/Dataset/archive (1).zip (Unzipped Files)'
train_dataset_path = '/content/drive/MyDrive/Machine Learning Project/Driver Inattention Detection/Dataset/archive (1).zip (Unzipped Files)/train'
valid_dataset_path = '/content/drive/MyDrive/Machine Learning Project/Driver Inattention Detection/Dataset/archive (1).zip (Unzipped Files)/valid'
test_dataset_path = '/content/drive/MyDrive/Machine Learning Project/Driver Inattention Detection/Dataset/archive (1).zip (Unzipped Files)/test'

In [4]:
#Image File Paths
train_images_path = os.path.join(train_dataset_path)
valid_images_path = os.path.join(valid_dataset_path)
test_images_path = os.path.join(test_dataset_path)

In [5]:
#Classes File Path
train_class_file_path = os.path.join(train_dataset_path, '_classes.txt')
valid_class_file_path = os.path.join(valid_dataset_path, '_classes.txt')
test_class_file_path = os.path.join(test_dataset_path, '_classes.txt')

In [6]:
#Annotations File Path
train_annotations_file_path = os.path.join(train_dataset_path, '_annotations.txt')
valid_annotations_file_path = os.path.join(valid_dataset_path, '_annotations.txt')
test_annotations_file_path = os.path.join(test_dataset_path, '_annotations.txt')

In [7]:
# Function to read class names from a file
def load_class_labels(class_file_path):
    with open(class_file_path, 'r') as file:
        classes = [line.strip() for line in file.readlines()]
    return classes

In [19]:
#Function to load annotation and mapping it with bounding box and class id's
def load_annotations(annotations_file_path):
    annotations = {}
    with open(annotations_file_path, 'r') as file:
        for line_number, line in enumerate(file, 1):  # Added line numbering for debugging
            line = line.strip()
            if not line:
                continue  # Skip empty lines
            parts = line.split()
            if len(parts) < 2:
                #print(f"Skipping line {line_number}: not enough parts. {parts[0]}")
                continue
            image_path = parts[0]
            bbox_and_class = parts[1].split(',')
            if len(bbox_and_class) != 5:
                print(f"Skipping line {line_number}: incorrect number of bbox elements.")
                continue
            try:
                bbox = list(map(int, bbox_and_class[:4]))
                class_id = int(bbox_and_class[4])
            except ValueError as e:
                print(f"Skipping line {line_number}: {e}")
                continue
            annotations[image_path] = class_id
            #images.append(load_image(image_path))
    return annotations

In [30]:
training_annotations = load_annotations(train_annotations_file_path)
training_annotations_df = pd.DataFrame(list(training_annotations.items()), columns=['Images', 'Class_Id'])
training_annotations_df

,Images,Class_Id
0,gB_8_s1_2019-03-11T15-01-33-01-00_ir_face_mp4-...,3
1,gA_2_s1_ir_face_mp4-219_jpg.rf.dc3b3b5a1ff934c...,3
2,gA_3_s1_ir_face_mp4-237_jpg.rf.dbaee33cd769e11...,2
3,gA_5_s2_ir_face_mp4-403_jpg.rf.dbaaf0a9f01069f...,0
4,gB_8_s1_2019-03-11T15-01-33-01-00_ir_face_mp4-...,3
...,...,...
11943,gB_6_s2_2019-03-11T13-46-14-01-00_ir_face_mp4-...,0
11944,gA_2_s2_ir_face_mp4-28_jpg.rf.db8c1df46f7865ba...,3
11945,gB_6_s1_2019-03-11T13-55-14-01-00_ir_face_mp4-...,1
11946,gA_4_s1_ir_face_mp4-220_jpg.rf.db8a753896cbffc...,3


In [11]:
#Convert images to Numpy Array
#np.array(images)